In [1]:
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datetime import datetime
from google.colab import drive
import os, zipfile, glob
import pickle

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Spanish books from the gutenberg.org for text generation (top 5 in volume with the right encoding)
'2000-0'
'62870-0'
'64465-0'
'6528-0'
'57654-0'

In [3]:
!wget "https://www.gutenberg.org/files/2000/2000-0.txt"

--2022-01-29 17:04:21--  https://www.gutenberg.org/files/2000/2000-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2226045 (2.1M) [text/plain]
Saving to: ‘2000-0.txt’

2000-0.txt          100%[===================>]   2.12M  4.41MB/s    in 0.5s    

2022-01-29 17:04:22 (4.41 MB/s) - ‘2000-0.txt’ saved [2226045/2226045]



In [4]:
!wget "https://www.gutenberg.org/files/62870/62870-0.txt"

--2022-01-29 17:04:22--  https://www.gutenberg.org/files/62870/62870-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1870046 (1.8M) [text/plain]
Saving to: ‘62870-0.txt’

62870-0.txt         100%[===================>]   1.78M  3.73MB/s    in 0.5s    

2022-01-29 17:04:23 (3.73 MB/s) - ‘62870-0.txt’ saved [1870046/1870046]



In [5]:
!wget "https://www.gutenberg.org/files/64465/64465-0.txt"

--2022-01-29 17:04:23--  https://www.gutenberg.org/files/64465/64465-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1397772 (1.3M) [text/plain]
Saving to: ‘64465-0.txt’

64465-0.txt         100%[===================>]   1.33M  3.24MB/s    in 0.4s    

2022-01-29 17:04:24 (3.24 MB/s) - ‘64465-0.txt’ saved [1397772/1397772]



In [6]:
!wget "https://www.gutenberg.org/files/6528/6528-0.txt"

--2022-01-29 17:04:27--  https://www.gutenberg.org/files/6528/6528-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4237055 (4.0M) [text/plain]
Saving to: ‘6528-0.txt’

6528-0.txt          100%[===================>]   4.04M  7.34MB/s    in 0.6s    

2022-01-29 17:04:27 (7.34 MB/s) - ‘6528-0.txt’ saved [4237055/4237055]



In [7]:
!wget "https://www.gutenberg.org/files/57654/57654-0.txt"

--2022-01-29 17:04:29--  https://www.gutenberg.org/files/57654/57654-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1131399 (1.1M) [text/plain]
Saving to: ‘57654-0.txt’

57654-0.txt         100%[===================>]   1.08M  2.62MB/s    in 0.4s    

2022-01-29 17:04:30 (2.62 MB/s) - ‘57654-0.txt’ saved [1131399/1131399]



In [8]:
!pip install unidecode

     |████████████████████████████████| 235 kB 9.0 MB/s 


In [9]:
import codecs
from bs4 import BeautifulSoup
import unidecode

In [10]:
list_books = ['2000-0', '62870-0', '64465-0', '6528-0', '57654-0']
text_add = ''
for b in list_books:
  with open(b + '.txt', 'r') as f:
    text = f.read()

    info_beg = text.find('Title: ')
    begin = text.find('*** START OF THIS PROJECT GUTENBERG EBOOK ')
    title = ''
    if begin > -1:
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 42 + len(title) + 3
                
    else:
      begin = text.find('START OF THE PROJECT GUTENBERG EBOOK ')
      for i in text[info_beg : begin].split('\n'):
        if i.find('Title: ') > -1:
          title = i.split('Title: ')[1]
          begin_2 = begin + 37 + len(title) + 3

    end = text.find('*** END OF THIS PROJECT GUTENBERG EBOOK')
    if end == -1:
      end = text.find('*** END')
    text = text[begin_2 : end]

    #remove html tags from text
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")
    
    #remove accented characters from text, e.g. café
    text = unidecode.unidecode(text)

    text_add = text_add + text + '\n'

In [11]:
len(text.split(' '))

172417

In [12]:
# Create two dictionaries:
# int2char -- maps integers to characters
# char2int -- maps characters to unique integers

int2char = dict(enumerate(set(text)))
char2int = { val: idx  for idx, val in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [13]:
len(char2int)

82

In [14]:
def one_hot_encode(arr, n_labels):
    
    # arr - array of integers
    # n_labels - number of labels (the size of a one-hot-encoded vector)

    one_hot = np.eye(n_labels)[arr]
    
    return one_hot

In [15]:
# check that the function works correctly
test_indx = np.array([[7, 2, 5]])
one_hot = one_hot_encode(test_indx, 8)

print(one_hot)

[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]


In [16]:
def get_batches(arr, batch_size, seq_length):
    # Create a generator that returns batches of size batch_size x seq_length
    
    batch_size_total = batch_size * seq_length
    ## Get the number of batches we can make
    n_batches = int(len(arr) / batch_size_total)
    
    ## Keep only enough characters to make full batches
    arr = arr[: n_batches*batch_size_total] 
    
    ## Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1)) 
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n : n + seq_length ]
        # The target is a version of x shifted by one (do not forget border conditions)
        y = np.zeros_like(x)
        y[:, :-1] = x[:, 1:]
        if n + seq_length < arr.shape[1]:
            y[:, -1] = arr[:, n + seq_length]
        else:
            arr[:, 0]
        yield x, y

In [17]:
batches = get_batches(encoded, 4, 30)
x, y = next(batches)

In [18]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU


In [19]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        ## YOUR CODE HERE
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=self.n_hidden, 
                            num_layers=self.n_layers, dropout=self.drop_prob, 
                            batch_first=True)

        # Define a dropout layer
        ## YOUR CODE HERE
        self.dropout = nn.Dropout(self.drop_prob)

        # Define the final, fully-connected output layer
        ## YOUR CODE HERE
        self.linear = nn.Linear(in_features=self.n_hidden, out_features=len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        ## YOUR CODE HERE
        out, hidden = self.lstm(x.float(), hidden)

        # Pass through a dropout layer
        ## YOUR CODE HERE
        out = self.dropout(out)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        ## YOUR CODE HERE
        out = out.reshape(out.size()[0] * out.size()[1], self.n_hidden)


        # Put x through the fully-connected layer
        ## YOUR CODE HERE
        out = self.linear(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden    

In [20]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                now = datetime.now()
                dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                      "Time: ", dt_string)

In [21]:
n_hidden = 512 
n_layers = 2

chars = tuple(set(text))
net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(82, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=82, bias=True)
)


In [22]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/new_dict_chars.pickle', 'wb') as f:
    pickle.dump(chars, f)

In [23]:
with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/new_dict_chars.pickle', 'rb') as f:
  c = pickle.load(f)

In [24]:
print(c)

('A', '0', 'Z', 'Q', '5', '\n', 'l', '[', 'K', 's', 'I', 'u', 'E', 'x', 'G', '?', 'z', '6', '<', '*', ' ', 'C', '8', 'H', '4', 'j', 't', 'R', 'h', 'm', 'P', '3', 'O', 'i', '/', '7', 'a', ',', 'V', 'c', 'p', ')', 'M', ';', '.', 'd', '!', 'k', 'f', 'y', "'", 'W', 'w', 'X', 'n', 'D', 'U', 'N', 'e', 'F', '1', 'B', '>', 'r', 'T', '9', 'o', ':', 'Y', ']', 'q', 'v', 'S', '-', '2', 'L', 'J', 'b', '"', '(', 'g', '_')


In [25]:
batch_size = 256
seq_length = 100 
n_epochs = 80

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=500)

Epoch: 14/80... Step: 500... Loss: 1.8864... Val Loss: 1.7935 Time:  29/01/2022 17:06:36
Epoch: 28/80... Step: 1000... Loss: 1.6097... Val Loss: 1.4722 Time:  29/01/2022 17:07:25
Epoch: 41/80... Step: 1500... Loss: 1.3666... Val Loss: 1.3412 Time:  29/01/2022 17:08:13
Epoch: 55/80... Step: 2000... Loss: 1.2512... Val Loss: 1.2769 Time:  29/01/2022 17:09:01
Epoch: 68/80... Step: 2500... Loss: 1.1633... Val Loss: 1.2759 Time:  29/01/2022 17:09:50


In [26]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/new_text_generator.pt" 
torch.save(net.state_dict(), path)


In [27]:
path = "/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/new_text_generator.pt" 


with open('/content/drive/MyDrive/2022-01-15_Course_project/LSTM_generator_es/new_dict_chars.pickle', 'rb') as f:
    chars_2 = pickle.load(f)

n_hidden = 512  
n_layers = 2
net_2 = CharRNN(chars_2, n_hidden, n_layers)
net_2.load_state_dict(torch.load(path))

<All keys matched successfully>

In [28]:
net_2.eval()

CharRNN(
  (lstm): LSTM(82, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=512, out_features=82, bias=True)
)

In [29]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [30]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [31]:
# Example of generated text
print(sample(net, 8000, prime='Ahora pase el personaje anterior y obtenga uno nuevo', top_k=20))

Ahora pase el personaje anterior y obtenga uno nuevo,
los enemigos y amenazadores, ni dibo las provercidas de
verguence con su amigo de igual vasta Hector, que padren a los
correntes cuantos cerca en la ciera, arranto la loniza o para
el mar librarando de la guerra. Ojola vrenejar al dios dinciendo
a Tidis, a quien te simo tiene valerosa vido, codo dia frente, menerio. Jumite
en la tierra, les exhorto en sus ojos en la parpes facilmente, y
dejale al eter y al rio en sus miguras mas ardelicieste.>>

442 Respondiole Hector, caro a Jupiter, que lleva la egida! !Con
ena que los demas fertines menos! Pero, dejandoso Aquiles, les
quitaron las filas cuando aquellas tomando los briylantes ojos,
como lo que se ocurate. Y anfian por las diosas
orados de buen dentido clavadas al peso veloz mas proponiose de
este que fueron muerto; cual destruido por su agito par tantos desapidas
mejores; y tan veloras semejantes alerados de lo que prentaparame
fagilando vengo al luchas companero Pona internada, y

In [32]:
text_ph = text.split('. ')

In [33]:
list_b1 = [ i.split(' ')[0].lower() for i in text_ph]
list_b2 = [ ' '.join(j.lower() for j in i.split(' ')[: 2]) for i in text_ph]
list_b3 = [ ' '.join(j.lower() for j in i.split(' ')[: 3]) for i in text_ph]
list_b4 = [ ' '.join(j.lower() for j in i.split(' ')[: 4]) for i in text_ph]

In [34]:
Counter(list_b1).most_common(10)

[('y', 331),
 ('el', 155),
 ('pero', 112),
 ('los', 81),
 ('como', 78),
 ('mas', 69),
 ('no', 66),
 ('en', 62),
 ('nombre', 59),
 ('a', 56)]

In [35]:
import re

In [36]:
list_of_begins = set()
for i in Counter(list_b1).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b2).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

for i in Counter(list_b3).most_common(10):
  list_of_begins.add(re.sub("[^A-Za-z] ", "", i[0]).capitalize())

In [37]:
list(list_of_begins)

['',
 'A',
 ' ',
 'El guerrero cayo',
 'Y en',
 'Eaobremos todos',
 'Mas ?por que',
 'Mas',
 'Y ahora',
 'Y el',
 'Nombre patronimico de\n',
 'Y',
 'Pero',
 'Y si',
 'Nombre patronimico de',
 'Fue muerto por',
 'Como el',
 'El',
 'El hijo de',
 'Como',
 'Los',
 'Pero no',
 'Y cuando',
 'No',
 'El atrida',
 'Nombre',
 'Nombre patronimico',
 'En']

In [38]:
# Removing the empty starting characters
list_of_begins.remove('')

In [39]:
# Removing the empty starting characters
list_of_begins.remove(' ')

In [40]:
list(list_of_begins)

['A',
 'El guerrero cayo',
 'Y en',
 'Eaobremos todos',
 'Mas ?por que',
 'Mas',
 'Y ahora',
 'Y el',
 'Nombre patronimico de\n',
 'Y',
 'Pero',
 'Y si',
 'Nombre patronimico de',
 'Fue muerto por',
 'Como el',
 'El',
 'El hijo de',
 'Como',
 'Los',
 'Pero no',
 'Y cuando',
 'No',
 'El atrida',
 'Nombre',
 'Nombre patronimico',
 'En']

In [41]:
import random

In [ ]:

num_docs = 10000

path = '/content/drive/MyDrive/2022-01-15_Course_project/new_gen_text_es/'

for i in range(num_docs):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')


29/01/2022 06:59:52 |  number of processed files: 0 0.0 %
29/01/2022 07:13:32 |  number of processed files: 200 2.0 %
29/01/2022 07:27:11 |  number of processed files: 400 4.0 %


In [42]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

# This is the code to generate (to determine how many have already been generated)

list_gen = glob.glob('/content/drive/MyDrive/2022-01-15_Course_project/new_gen_text_es/*.txt')
list_num = [ int(i.split('/')[-1].split('.txt')[0].split('doc_')[-1]) for i in list_gen]
max(list_num), min(list_num)


(5883, 0)

In [43]:
# If the generator does not work completely, you need to run this code (it will start from the last place)

num_docs = 10000

path = '/content/drive/MyDrive/2022-01-15_Course_project/new_gen_text_es/'

for i in range(max(list_num), num_docs, 1):
  start = random.choice(list(list_of_begins))
  text_gen = sample(net, 8000, prime= start, top_k=10)

  with open(path + 'doc_'+str(i)+'.txt', 'w+') as output_file:
    try:
      output_file.write(text_gen)
    except:
      continue
  if i % 200 == 0:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string, '| ',  'number of processed files: ' + str(i), round(i/num_docs, 3)*100, '%')

29/01/2022 17:29:20 |  number of processed files: 6000 60.0 %
29/01/2022 17:42:47 |  number of processed files: 6200 62.0 %
29/01/2022 17:56:13 |  number of processed files: 6400 64.0 %
29/01/2022 18:09:39 |  number of processed files: 6600 66.0 %
29/01/2022 18:23:07 |  number of processed files: 6800 68.0 %
29/01/2022 18:36:35 |  number of processed files: 7000 70.0 %
29/01/2022 18:50:04 |  number of processed files: 7200 72.0 %
29/01/2022 19:03:32 |  number of processed files: 7400 74.0 %
29/01/2022 19:17:00 |  number of processed files: 7600 76.0 %
29/01/2022 19:30:26 |  number of processed files: 7800 78.0 %
29/01/2022 19:43:51 |  number of processed files: 8000 80.0 %
29/01/2022 19:57:16 |  number of processed files: 8200 82.0 %
29/01/2022 20:10:41 |  number of processed files: 8400 84.0 %
29/01/2022 20:24:06 |  number of processed files: 8600 86.0 %
29/01/2022 20:37:31 |  number of processed files: 8800 88.0 %
29/01/2022 20:50:56 |  number of processed files: 9000 90.0 %
29/01/20